## Atomic Swaps
#### 04.5 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2021-11-28

* Transaction groups
* Atomic swap
* **Requires** 04.3_WSC_Token including exercises

### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [16]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo = cred['MyAlgo']
Alice  = cred['Alice']
Bob    = cred['Bob']
Charlie = cred['Charlie']

In [17]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error

import json
import pandas as pd
import base64

In [18]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()['last-round']

27734266

## ❗️Atomic Swap

Trade ("swap") one asset for another one: very important economic activity

* TradFi: need trusted intermediary
* Algorand blockchain: implemented as an Atomic Transfer
    * Idea: either *all* transactions succeed or *all* fail
    * Transactions can contain ALOGs or ASA
    * Possible: transactions with more than 2 parties

### A concrete Atomic Swap
"Alice buys 2.5 WSC from Bob for 3.75 Algos"
* **Transaction 1**: Alice sends Bob 3.75 Algos
* **Transaction 2**: Bob sends Alice 2.5 WSC

#### Step 0: Get the status before the swap
* For information only
* Compare holdings of Alice and Bob using `asset_holdings_df2`

In [19]:
asset_holdings_df2(algod_client, Alice['public'], Bob['public'], suffix=['Alice','Bob'])

,amountAlice,unit,asset-id,name,decimals,amountBob
0,24.132,ALGO,0,Algorand,6,2.383000e+00
1,180.000,USDC,10458941,USDC,6,2.000000e+01
2,NaN,Beer,159171974,Beer Coin,2,3.010002e+07
3,NaN,TEMP,159173248,Peters Tempcoin,1,8.330000e+01
4,NaN,TEMPYRY,159173586,Pyrys Tempcoin,1,8.300000e+00
5,NaN,WSC,159189398,Peters WSC coin,2,1.000000e+01


In [20]:
# Store the correct ID for the WSC coin from 04.3_WSC_Token
WSC_id=10458941                              # <---------- Update!!

#### Step 1a: Prepare transaction 1
ALGO payment. Alice sends Bob 3.75 ALGOs

In [21]:
sp = algod_client.suggested_params()
amt_1 = int( 3.75*1E6 )                     # microalgos!

txn_1 = PaymentTxn(Alice["public"], sp, Bob["public"],amt_1)

#### Step 1b: Prepare transaction 2
ASA transfer. Bob transfers 2.5 WSC coungs to Alice.<br>
Alice has to opt into the WSC coin (In our case, she did so in notebook 04.1!)

In [22]:
amt_2 = int(2.5 * 1E2)                      # WSC coin is 1/100 divisible !!

txn_2 = AssetTransferTxn(Bob["public"], sp, Alice["public"], amt_2, WSC_id)

#### Step 1c: create a TX group
* Watch the `group_id` ... it is going to be very important

In [23]:
# group_id calculated from list of transactions
gid = transaction.calculate_group_id([txn_1, txn_2])

# add group_id to each transactions
txn_1.group = gid
txn_2.group = gid

# This is the gid (for info only)
print( base64.b32encode(gid).decode() )

KYY52NZFJEECYJY6GYKOFXRIXVQWV4UMRZ47OZKAPADJLROY4CQA====


#### Step 2: Sign
* Everyone has to sign his/her transaction
* Signing happens *after* adding group ID
    * You sign the TX
    * You also sign the fact that it is part of a group

In [24]:
# sign transactions
stxn_1 = txn_1.sign(Alice["private"])    
stxn_2 = txn_2.sign(Bob["private"])

#### Step 3: Assemble and send

In [25]:
# assemble transaction group
signed_group =  [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

#### Step 4: Wait for confirmation

In [26]:
# wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid) 

Current round is  27734271.
Waiting for round 27734271 to finish.
Transaction WYB34TGEY6K6Q5NS7L6GTUO6ARYLQSNBUUPAQCGF5LNLD3Q4VDUA confirmed in round 27734272.


#### Step 5: Check holdings after swap

In [27]:
asset_holdings_df2(algod_client, Alice['public'], Bob['public'], suffix=['Alice','Bob'])

,amountAlice,unit,asset-id,name,decimals,amountBob
0,20.38100,ALGO,0,Algorand,6,6.132000e+00
1,180.00025,USDC,10458941,USDC,6,1.999975e+01
2,NaN,Beer,159171974,Beer Coin,2,3.010002e+07
3,NaN,TEMP,159173248,Peters Tempcoin,1,8.330000e+01
4,NaN,TEMPYRY,159173586,Pyrys Tempcoin,1,8.300000e+00
5,NaN,WSC,159189398,Peters WSC coin,2,1.000000e+01


## Discussion: the relevance of the transaction groups
* Implement an "all or nothing" logic
* If **only one** transaction fails, the **whole transaction group** fails ... 
    * If one account would be overspending
    * If one signature is not valid
    * If one account has not opted in
    * If one transaction has been modified after calulating the `group_id`

## Appendix: how to merge dataframes
This section explains how `asset_holdings_df2()`works

* The Python library for working with dataframes is called Pandas
* Most people abbreviate it to `pd` using `import pandas as pd`
* The `pd.merge()` method merges two dataframes ... but how exactly?

In [28]:
# get the holdings of Alice and Bob separately
alice_holding=asset_holdings_df(algod_client, Alice['public'])
bob_holding=asset_holdings_df(algod_client, Bob['public'])

In [29]:
# Merge in one data.frame using pandas merge
pd.merge(alice_holding, bob_holding,  
         how="outer", 
         on=["asset-id", "unit", "name", "decimals"], 
         suffixes=['Alice','Bob'])

,amountAlice,unit,asset-id,name,decimals,amountBob
0,20.38100,ALGO,0,Algorand,6,6.132000e+00
1,180.00025,USDC,10458941,USDC,6,1.999975e+01
2,NaN,Beer,159171974,Beer Coin,2,3.010002e+07
3,NaN,TEMP,159173248,Peters Tempcoin,1,8.330000e+01
4,NaN,TEMPYRY,159173586,Pyrys Tempcoin,1,8.300000e+00
5,NaN,WSC,159189398,Peters WSC coin,2,1.000000e+01


In [30]:
# package as function
def asset_holdings_df2(client,adr1,adr2,suffix=['','']):
    # client = algosdk client
    # adr1, adr2 = public address to be analyzed
    import pandas as pd
    from algosdk.v2client import algod
    info1 = asset_holdings(client, adr1)
    df1 = pd.DataFrame(info1)
    info2 = asset_holdings(client, adr2)
    df2 = pd.DataFrame(info2)
    df_merge = pd.merge(df1,df2,how="outer", on=["asset-id", "unit", "name", "decimals"],suffixes=suffix)
    return(df_merge)